# Filtering

In [68]:
import pandas as pd
import os

songs_df = pd.read_csv(os.path.join("data", "songs_v1.csv"))
print("Length before filtering: ", len(songs_df))

Length before filtering:  10854


1. Drop duplicates:

In [69]:
songs_df = songs_df.drop_duplicates(subset=["title", "artist"])
print("Length after dropping duplicates: ", len(songs_df))

Length after dropping duplicates:  10804


2. Filter non ukr artists and dublicates of the same artist:

In [70]:
ARTISTS_BLACKLIST = [
    "Poshlaya Molly", "Poshlaja Molli", "Сметана band", "Corn Wave", "Потап И Настя Каменских", 
    "Светлана Лобода", "Valentin Strykalo", "Верка Сердючка", "Lюk", "kavabanga Depo kolibri", 
    "Quest Pistols Show", "Какая Разница", "Green Grey", "jockii druce", "Миша Марвин", 
    "Оля Полякова", "Михаил Бублик", "Алёна Винницкая", "Наталья Могилевская",
    "Таисия Повалий", "Сергій Жадан Та Собаки В Космосі", "Poshlaya Molly",
    "Valentin Strykalo", "INGRET", "Svitlana Nianio", "Vasily Richter",
    "MARUV & Boosin", "Сусіди Стерплять", "The Feels", "Grandma's smuzi", 
    "Табула Раса", "Олег Кензов", "Potap & Nastya", "Виталий Чирва", "TELLY GRAVE",
    "Анастасия Приходько", "glichery", "AShamaluevMusic", "daKooka", "4Wheel", "KLIM", "CLONNEX", 
    "bris", "Pencil Legs", "Mykola Dmytrovych Leontovych", "Odyn v kanoe", 
    "Святослав Вакарчук", "DJ Jedy", "Марія Чайковська", "The Ukrainians", "Los Colorados", 
    "Тіна Кароль", "Ірина Білик", "national radio"
    ]

songs_df = songs_df[~songs_df["artist"].isin(ARTISTS_BLACKLIST)]
print("Length after removing blacklisted artists: ", len(songs_df))

Length after removing blacklisted artists:  5872


3. Remove songs that have brackets in the title (probably remixes of the original songs) or are too long:

In [71]:
songs_df = songs_df[~songs_df["title"].str.contains("\(") | ~songs_df["title"].str.contains("\)")]
songs_df = songs_df[~songs_df["title"].str.contains("\[") | ~songs_df["title"].str.contains("\]")]
songs_df = songs_df[~songs_df["title"].str.contains("\|")]
songs_df = songs_df[~songs_df["title"].str.contains("\.")]
songs_df = songs_df[songs_df["title"].str.len() < 30]

print("Length after filtering based on song title: ", len(songs_df))

Length after filtering based on song title:  4417


4. Filter non ukr artists based on the songs' tags:

Note: don't include russian here as a considerable amount of ukr songs have a missleading russian tag

In [72]:
TAGS_BLACKLIST = ["finnish", "japanese"]
songs_df = songs_df[~songs_df["title_tags"].str.contains("|".join(TAGS_BLACKLIST))]
print("Length after filtering based on title tags: ", len(songs_df))

Length after filtering based on title tags:  4341


Save the filtered songs to a new *.csv file:

In [ ]:
songs_df.to_csv(os.path.join("data", "songs_filtered_v1.csv"), index=False)

# Data summary after filtering

In [93]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "songs_filtered_v1.csv"))

In [97]:
print("Number of ukr artists: ", len(filtered_songs_df["artist"].unique()))

Number of ukr artists:  62


In [98]:
songs_per_artist = filtered_songs_df["artist"].value_counts()

print(f"Total number of songs: {len(filtered_songs_df)}\n")
print(f"Number of songs per artist:\n{songs_per_artist.describe()}")

Total number of songs: 4341

Number of songs per artist:
count     62.000000
mean      70.016129
std       21.063909
min       18.000000
25%       53.000000
50%       75.500000
75%       87.750000
max      100.000000
Name: artist, dtype: float64


Save the current blacklist version to a *.txt file":

In [ ]:
with open(os.path.join("configs", "artists_blacklist_v1.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(ARTISTS_BLACKLIST))

# Concat with YT fetched results

In [ ]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "songs_filtered_v1.csv"))
len(filtered_songs_df)

4341

In [ ]:
audios_5_df = pd.read_csv(os.path.join("metadata", "yt_songs_5_2_pages_filtered.csv"))
audios_10_df = pd.read_csv(os.path.join("metadata", "yt_songs_10_2_pages_filtered.csv"))
audios_more_df = pd.read_csv(os.path.join("metadata", "yt_songs_more_sooongs_filtered.csv"))

audios_df = pd.concat([audios_5_df, audios_10_df, audios_more_df], ignore_index=True)

print("Total: ", len(audios_df))
print("No yt_url: ", len(audios_df[audios_df["yt_url"].isnull()]))

Total:  4068
No yt_url:  602


In [89]:
merged_df = pd.merge(filtered_songs_df, audios_df, on=["title", "artist"], how="left")

print("Columns: ", merged_df.columns.values)
print("Total: ", len(merged_df))
print("No yt_url: ", len(merged_df[merged_df["yt_url"].isnull()]))

Columns:  ['title' 'artist' 'title_listeners_x' 'title_scrobbles_x' 'title_tags_x'
 'title_duration_x' 'title_listeners_y' 'title_scrobbles_y' 'title_tags_y'
 'title_duration_y' 'yt_title' 'yt_url' 'yt_duration' 'yt_views'
 'audio_path']
Total:  4341
No yt_url:  886


In [ ]:
merged_df[["title", "artist", "audio_path", "yt_title", "yt_url", "yt_duration", "yt_views"]]\
    .to_csv(os.path.join("data", "yt_songs_filtered_v1.csv"), index=False)

Updated summary after fetching the missing songs:

In [3]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "yt_songs_filtered_v1.csv"))

print("Columns: ", filtered_songs_df.columns.values)
print("Total: ", len(filtered_songs_df))
print("No yt_url: ", len(filtered_songs_df[filtered_songs_df["audio_path"].isnull()]))

Columns:  ['title' 'artist' 'audio_path' 'yt_title' 'yt_url' 'yt_duration'
 'yt_views']
Total:  4341
No yt_url:  60


# Exploring non unique audio_path

In [38]:
import pandas as pd
import os

filtered_songs_df = pd.read_csv(os.path.join("data", "yt_songs_filtered_v1.csv"))

len(filtered_songs_df["audio_path"].unique())

3227

In [8]:
non_unique_paths = filtered_songs_df["audio_path"].value_counts()[filtered_songs_df["audio_path"].value_counts() > 1].index.tolist()
non_unique_paths

['audio\\Нумер 482 - Добрий ранок Україно - (Офіційний кліп- 2015).mp3',
 'audio\\Ігор Кайдаш - цього вартує кохання (official mood video).mp3',
 'audio\\Димна Сумiш - Зламанi.mp3',
 'audio\\Серцевий Напад - Субкультура.mp3',
 'audio\\Нумер 482 - Важлива | Official Video.mp3',
 'audio\\Крихітка Цахес - На Першому Місці.mp3',
 "audio\\Її душі зів'ялі квіти.mp3",
 'audio\\◾Плач Єремії ◾ Вона ◾.mp3',
 'audio\\Лента За Лентою.mp3',
 'audio\\Лінія Маннергейма – Де твоя лінія?.mp3',
 'audio\\Холодне Сонце - Тінь Кохання (remastered HQ).mp3',
 'audio\\Один в каное - Подобається, як ти ідеш.mp3',
 'audio\\Воплі Відоплясова - ВЕСНА.mp3',
 'audio\\Воплі Відоплясова - Були на селі.mp3',
 'audio\\Катя Chilly - Я - молодая [HD 720p].mp3',
 'audio\\Плач Єремії - Ти втретє цього літа зацвітеш.mp3',
 'audio\\Sashakhuy.mp3',
 'audio\\Ало-але.mp3',
 'audio\\Zalyshai.mp3',
 'audio\\Воплі Відоплясова - Сонячні дні [Official Video].mp3',
 'audio\\The Unsleeping — PTAHA FRED (Official Video).mp3',
 'audio\\

It seems like we have to cases:
1. It is literaly the same song but with slightly different title name in the LastFM platform

In [9]:
filtered_songs_df[filtered_songs_df["audio_path"] == 'audio\\Ало-але.mp3']

,title,artist,audio_path,yt_title,yt_url,yt_duration,yt_views
752,Ало-але,Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172200.0
805,Ало але,Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0
812,"Алло, але",Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0
813,"Твої ало, мої але",Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0
818,"Ало, але",Крихітка Цахес,audio\Ало-але.mp3,Ало-але,https://youtube.com/watch?v=xH6cxIXtGWI,222.0,172201.0


2. Those are different songs, but the script failed to search the relevant YT audio (in some cases it doesn't even exist on YT)

In [36]:
filtered_songs_df[filtered_songs_df["audio_path"] == "audio\Крихітка Цахес - На Першому Місці.mp3"]

,title,artist,audio_path,yt_title,yt_url,yt_duration,yt_views
750,На першому місці,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
771,Ти на першому місці,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
777,На першому місці,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
782,На першому мiсцi,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102050.0
800,Ty na pershomu misci,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102051.0
801,На першому місті,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102051.0
814,На першому місті,Крихітка Цахес,audio\Крихітка Цахес - На Першому Місці.mp3,Крихітка Цахес - На Першому Місці,https://youtube.com/watch?v=RT6BXmAArYc,212.0,102051.0


We need to perform two further postprocessing steps:
1. Normalize artist and titles and filter duplicates
2. Prepare a column which will indicate the need for re-downloading the audio (or setting None if it doesn't exist on YT)

In [19]:
non_unique_paths = filtered_songs_df.groupby("title").filter(lambda x: x["artist"].nunique() > 1)["audio_path"].value_counts()
non_unique_paths = non_unique_paths[non_unique_paths > 1].index.tolist()
non_unique_paths

['audio\\Воплі Відоплясова - Були на селі.mp3',
 'audio\\Воплі Відоплясова - ВЕСНА.mp3',
 'audio\\Їхали козаки.mp3',
 'audio\\Там, Під Львівським Замком.mp3',
 'audio\\Intro.mp3',
 'audio\\Пачка цигарок.mp3',
 'audio\\Жадан і Собаки feat. Gogol Bordello – 5-а авеню (Офіційне відео).mp3',
 'audio\\Воплі Відоплясова - Сонячні дні [Official Video].mp3',
 'audio\\Воплі Відоплясова - Зоряна осінь.mp3',
 'audio\\Кедь Ми Пришла Карта.mp3',
 'audio\\Воплі Відоплясова - Танцi.mp3',
 'audio\\Я підійду.mp3',
 'audio\\Тартак feat. Катя Chilly — Понад Хмарами....mp3',
 'audio\\Трава.mp3',
 'audio\\Глибина.mp3',
 'audio\\Плач Єремії - Ти втретє цього літа зацвітеш.mp3',
 'audio\\Буде Нам З Тобою Що Згадати....mp3',
 'audio\\Олександр Пономарьов "Сам собі країна" - Скрябін. Концерт пам\'яті.mp3',
 'audio\\Жадан і Собаки, Христина Соловій – Серце (Офіційне відео).mp3',
 'audio\\Ми - хлопці з Бандерштадту.mp3',
 'audio\\Вона.mp3',
 'audio\\Я твоє крило.mp3',
 'audio\\Жадан і Собаки – Автозак (Офіційне 

In [33]:
antitila_df = filtered_songs_df[filtered_songs_df["audio_path"] == 'audio\\Я підійду.mp3']
antitila_df

,title,artist,audio_path,yt_title,yt_url,yt_duration,yt_views
627,Я підійду,Воплі Відоплясова,audio\Я підійду.mp3,Я підійду,https://youtube.com/watch?v=eBTkMoyZgp0,137.0,31225.0
643,Я пiдiйду,Воплі Відоплясова,audio\Я підійду.mp3,Я підійду,https://youtube.com/watch?v=eBTkMoyZgp0,137.0,31225.0
1380,Я пiдiйду,Воплi Вiдоплясова,audio\Я підійду.mp3,Я підійду,https://youtube.com/watch?v=eBTkMoyZgp0,137.0,31226.0
1385,Я підійду,Воплi Вiдоплясова,audio\Я підійду.mp3,Я підійду,https://youtube.com/watch?v=eBTkMoyZgp0,137.0,31226.0


In [34]:
antitila_df["artist"].value_counts()

Воплі Відоплясова    2
Воплi Вiдоплясова    2
Name: artist, dtype: int64

In [35]:
antitila_df["title"].value_counts()

Я підійду    2
Я пiдiйду    2
Name: title, dtype: int64